In [ ]:
!pip install fiftyone datasets


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 113.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.9/110.9 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 30.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.4/112.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.8/74.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 76.3 MB/s e

In [ ]:
import os
import json
import numpy as np
from PIL import Image
import fiftyone as fo

dataset = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="train",     # alidation/test
    label_types=["segmentations"],
    classes=["Pig"],
    max_samples=1024,
    dataset_name="open-images-pig",
    only_matching=True
)

dataset_2 = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="train",     # alidation/test
    label_types=["segmentations"],
    classes=["Horse"],
    max_samples=1024,
    dataset_name="open-images-horse",
    only_matching=True
)

dataset_3 = fo.zoo.load_zoo_dataset(
    "open-images-v7",
    split="train",     # alidation/test
    label_types=["segmentations"],
    classes=["Sheep"],
    max_samples=1024,
    dataset_name="open-images-sheep",
    only_matching=True
)

dataset.merge_samples(dataset_2)
dataset.merge_samples(dataset_3)

root_dir = "dataset"
imgs_dir = os.path.join(root_dir, "imgs")
masks_dir = os.path.join(root_dir, "masks")
os.makedirs(imgs_dir, exist_ok=True)
os.makedirs(masks_dir, exist_ok=True)

data = []

for sample in dataset:
    img_src = sample.filepath
    img_name = os.path.basename(img_src)
    img_dst = os.path.join(imgs_dir, img_name)

    if not os.path.exists(img_dst):
        os.system(f'cp "{img_src}" "{img_dst}"')

    detections = sample["ground_truth"].detections

    for i, det in enumerate(detections):
        label = det.label
        mask = det.mask

        mask_filename = f"{os.path.splitext(img_name)[0]}_{i}.png"
        mask_path = os.path.join(masks_dir, mask_filename)

        mask_img = Image.fromarray(mask.astype(np.uint8) * 255)
        mask_img.save(mask_path)

        data.append({
            "img_path": f"imgs/{img_name}",
            "mask_path": f"masks/{mask_filename}",
            "label": label
        })

json_path = os.path.join(root_dir, "data.json")
with open(json_path, "w") as f:
    json.dump(data, f, indent=2)


INFO:fiftyone.zoo.datasets:Downloading split 'train' to '/root/fiftyone/open-images-v7/train' if necessary


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/2018_04/train/train-images-boxable-with-rotation.csv' to '/root/fiftyone/open-images-v7/train/metadata/image_ids.csv'


 100% |██████|    4.8Gb/4.8Gb [15.5s elapsed, 0s remaining, 319.2Mb/s]      


INFO:eta.core.utils: 100% |██████|    4.8Gb/4.8Gb [15.5s elapsed, 0s remaining, 319.2Mb/s]      


INFO:fiftyone.utils.openimages:Downloading 'https://storage.googleapis.com/openimages/v5/class-descriptions-boxable.csv' to '/root/fiftyone/open-images-v7/train/metadata/classes.csv'


...

INFO:fiftyone.zoo.datasets:Loading 'open-images-v7' split 'train'


 100% |█████████████████| 931/931 [38.2s elapsed, 0s remaining, 24.8 samples/s]      


INFO:eta.core.utils: 100% |█████████████████| 931/931 [38.2s elapsed, 0s remaining, 24.8 samples/s]      


Dataset 'open-images-sheep' created


INFO:fiftyone.zoo.datasets:Dataset 'open-images-sheep' created


In [ ]:
import json

data = json.load(open("/content/dataset/data.json", "rt"))

In [ ]:
len(data)
count_ = {
    "Pig": 0,
    "Horse": 0,
    "Sheep": 0
}

for item in data:
  count_[item["label"]] += 1

count_

{'Pig': 1082, 'Horse': 1822, 'Sheep': 2051}

In [ ]:
! zip dataset.zip dataset -r

updating: dataset/ (stored 0%)
  adding: dataset/masks/ (stored 0%)
  adding: dataset/masks/2d6b1516e65cd839_6.png (stored 0%)
  adding: dataset/masks/1f14c5be457340ab_0.png (deflated 4%)
  adding: dataset/masks/0139f4495914ca85_0.png (stored 0%)
  adding: dataset/masks/6fbb556ef8e90912_0.png (deflated 8%)
  adding: dataset/masks/47d70b71b4a37b3a_2.png (stored 0%)
  adding: dataset/masks/3b12acae2a2e2176_2.png (deflated 12%)
  adding: dataset/masks/5bedeb2529c0141f_0.png (stored 0%)
  adding: dataset/masks/18bb249d5d4014cb_0.png (stored 0%)
  adding: dataset/masks/44a4e21f548d3c8f_0.png (deflated 3%)
  adding: dataset/masks/871bf5d64a7d1340_0.png (stored 0%)
  adding: dataset/masks/0397a48d901636d5_0.png (stored 0%)
  adding: dataset/masks/418b364d8b04bc29_9.png (stored 0%)
  adding: dataset/masks/069a650ec355338d_2.png (stored 0%)
  adding: dataset/masks/b0b2a12418a4ac24_0.png (deflated 5%)
  adding: dataset/masks/18fcac4ba000dc9e_0.png (deflated 8%)
  adding: dataset/masks/189ff9a4a7

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
!mv dataset.zip /content/drive/MyDrive/


In [ ]:
!rm -rf dl-first-task
!rm -rf src

!git clone https://github.com/BioMikeUkr/dl-first-task.git


Cloning into 'dl-first-task'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (11/11), done.
Receiving objects: 100% (15/15), 9.01 KiB | 9.01 MiB/s, done.
Resolving deltas: 100% (3/3), done.
remote: Total 15 (delta 3), reused 11 (delta 3), pack-reused 0 (from 0)
